In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from dataclasses import dataclass
from typing import List, Dict, Tuple
from itertools import combinations
from abc import ABC, abstractmethod
import warnings
warnings.filterwarnings('ignore')

# ============================================================
#                    CONFIGURATION
# ============================================================

@dataclass
class GameConfig:
    """Configuration pour différents jeux."""
    name: str
    num_max: int
    nums_per_draw: int
    filepath: str

CONFIGS = {
    "loto": GameConfig("Loto", 49, 5, "lotodata.csv"),
    "keno": GameConfig("Keno", 70, 20, "kenodata.csv"),
    "euromillions": GameConfig("EuroMillions", 50, 5, "euromillions.csv"),
}

# ============================================================
#              FONCTIONS D'ACTIVATION
# ============================================================

def sigmoid(x):
    """Fonction sigmoid avec protection contre overflow."""
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    """Dérivée de la fonction sigmoid."""
    return x * (1 - x)

def softmax(x):
    """Fonction softmax stable numériquement."""
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum()

# ============================================================
#     MODULE RÉSEAU DE NEURONES (VARIANTE 1 - SGD)
# ============================================================

class NeuralNetworkSGD:
       
    def __init__(self, 
                 input_size: int = 70,
                 hidden_layers: List[int] = [128, 64],
                 output_size: int = 70,
                 learning_rate: float = 0.001):
        
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        # Initialisation des poids et biais
        self.weights = []
        self.biases = []
        
        layer_sizes = [input_size] + hidden_layers + [output_size]
        
        for i in range(len(layer_sizes) - 1):
            # Initialisation Xavier/Glorot
            limit = np.sqrt(6 / (layer_sizes[i] + layer_sizes[i+1]))
            w = np.random.uniform(-limit, limit, (layer_sizes[i], layer_sizes[i+1]))
            b = np.zeros((1, layer_sizes[i+1]))
            self.weights.append(w)
            self.biases.append(b)
        
        # Stockage des activations pour la rétropropagation
        self.activations = []
        self.z_values = []
        
    def forward(self, X: np.ndarray) -> np.ndarray:
        """Propagation avant."""
        self.activations = [X]
        self.z_values = []
        
        current = X
        for i, (w, b) in enumerate(zip(self.weights, self.biases)):
            z = np.dot(current, w) + b
            self.z_values.append(z)
            
            if i == len(self.weights) - 1:
                # Dernière couche: sigmoid pour probabilités
                current = sigmoid(z)
            else:
                # Couches cachées: sigmoid
                current = sigmoid(z)
            
            self.activations.append(current)
        
        return current
    
    def backward(self, y_true: np.ndarray, epsilon: float = 1e-7):
        """
        Rétropropagation avec SGD.
        Implémente la logique de Retropropagation_sgd du code FreeBasic.
        """
        m = y_true.shape[0] if len(y_true.shape) > 1 else 1
        y_pred = self.activations[-1]
        
        # Erreur de la couche de sortie (cross-entropy derivative)
        # erreurSortie = -cibles/sortie + (1-cibles)/(1-sortie)
        delta = (-y_true / (y_pred + epsilon) + (1 - y_true) / (1 - y_pred + epsilon))
        delta = delta * sigmoid_derivative(y_pred)
        
        deltas = [delta]
        
        # Rétropropagation à travers les couches cachées
        for i in range(len(self.weights) - 2, -1, -1):
            delta = np.dot(deltas[-1], self.weights[i + 1].T)
            delta = delta * sigmoid_derivative(self.activations[i + 1])
            deltas.append(delta)
        
        deltas.reverse()
        
        # Mise à jour des poids et biais
        for i in range(len(self.weights)):
            activation = self.activations[i]
            if len(activation.shape) == 1:
                activation = activation.reshape(1, -1)
            if len(deltas[i].shape) == 1:
                deltas[i] = deltas[i].reshape(1, -1)
                
            self.weights[i] -= self.learning_rate * np.dot(activation.T, deltas[i]) / m
            self.biases[i] -= self.learning_rate * np.mean(deltas[i], axis=0, keepdims=True)
    
    def compute_loss(self, y_true: np.ndarray, y_pred: np.ndarray, epsilon: float = 1e-7) -> float:
        """
        Calcul de la perte (cross-entropy binaire).
        perte = -cibles * log(sortie) - (1-cibles) * log(1-sortie)
        """
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        return loss
    
    def train(self, X: np.ndarray, y: np.ndarray, epochs: int = 100, verbose: bool = False):
        """Entraînement du réseau."""
        losses = []
        
        for epoch in range(epochs):
            # Propagation avant
            y_pred = self.forward(X)
            
            # Calcul de la perte
            loss = self.compute_loss(y, y_pred)
            losses.append(loss)
            
            # Rétropropagation
            self.backward(y)
            
            if verbose and (epoch + 1) % 10 == 0:
                print(f"   Epoch {epoch+1}/{epochs} - Loss: {loss:.6f}")
        
        return losses


# ============================================================
#     CALCULATEUR DE FEATURES (VARIANTE 1)
# ============================================================

class FeatureCalculator:
    """
    Calcule les features d'entrée pour le réseau de neurones.
    Traduit depuis le code FreeBasic.
    
    Features:
    1. Données historiques normalisées
    2. Fréquence relative des numéros
    3. "Déjà vu" (écart depuis dernière apparition)
    4. Moyennes mobiles
    5. Entropie des numéros
    """
    
    def __init__(self, config: GameConfig, 
                 coef1: float = 0.2,  # Données historiques
                 coef2: float = 0.25, # Fréquence relative
                 coef3: float = 0.2,  # Déjà vu
                 coef4: float = 0.15, # Moyenne mobile
                 coef5: float = 0.2): # Entropie
        
        self.config = config
        self.coef1 = coef1
        self.coef2 = coef2
        self.coef3 = coef3
        self.coef4 = coef4
        self.coef5 = coef5
        
    def compute_frequency(self, historique: List[List[int]]) -> np.ndarray:
        """Calcule la fréquence relative de chaque numéro."""
        freq = np.zeros(self.config.num_max)
        total = 0
        
        for draw in historique:
            for num in draw:
                if 1 <= num <= self.config.num_max:
                    freq[num - 1] += 1
                    total += 1
        
        if total > 0:
            freq = freq / total
        
        return freq
    
    def compute_dejavu(self, historique: List[List[int]]) -> np.ndarray:
        """
        Calcule le 'déjà vu' - nombre de tirages depuis la dernière apparition.
        Plus la valeur est élevée, plus le numéro est "en retard".
        """
        dejavu = np.zeros((len(historique), self.config.num_max))
        
        for tirage_idx in range(len(historique)):
            if tirage_idx > 0:
                # Incrémenter tous les déjà vu
                dejavu[tirage_idx] = dejavu[tirage_idx - 1] + 1
            
            # Réinitialiser pour les numéros tirés
            for num in historique[tirage_idx]:
                if 1 <= num <= self.config.num_max:
                    dejavu[tirage_idx, num - 1] = 0
        
        return dejavu
    
    def compute_moving_average(self, historique: List[List[int]], window: int = 10) -> np.ndarray:
        """Calcule la moyenne mobile des apparitions."""
        n_tirages = len(historique)
        ma = np.zeros((n_tirages, self.config.num_max))
        
        # Créer une matrice binaire d'apparition
        presence = np.zeros((n_tirages, self.config.num_max))
        for i, draw in enumerate(historique):
            for num in draw:
                if 1 <= num <= self.config.num_max:
                    presence[i, num - 1] = 1
        
        # Calculer la moyenne mobile
        for i in range(n_tirages):
            start = max(0, i - window + 1)
            ma[i] = np.mean(presence[start:i+1], axis=0)
        
        return ma
    
    def compute_entropy(self, historique: List[List[int]]) -> np.ndarray:
        """
        Calcule l'entropie pour chaque numéro.
        entropie = -p * log(p) où p est la probabilité d'apparition.
        """
        freq = self.compute_frequency(historique)
        entropy = np.zeros(self.config.num_max)
        
        for i in range(self.config.num_max):
            if freq[i] > 0:
                entropy[i] = -freq[i] * np.log(freq[i] + 1e-10)
        
        return entropy
    
    def compute_all_features(self, historique: List[List[int]]) -> Tuple[np.ndarray, np.ndarray]:
        """
        Calcule toutes les features et retourne les entrées et cibles.
        
        Returns:
            X: Matrice des entrées (n_tirages, num_max)
            y: Matrice des cibles (n_tirages, num_max)
        """
        n_tirages = len(historique)
        
        # Calculer chaque feature
        freq_relative = self.compute_frequency(historique)
        dejavu = self.compute_dejavu(historique)
        moving_avg = self.compute_moving_average(historique)
        entropy = self.compute_entropy(historique)
        
        # Normalisation de l'entropie
        entropy_max = np.max(entropy) if np.max(entropy) > 0 else 1
        entropy_norm = entropy / entropy_max
        
        # Construire la matrice d'entrée
        X = np.zeros((n_tirages, self.config.num_max))
        y = np.zeros((n_tirages, self.config.num_max))
        
        # Matrice binaire des tirages (pour coef1)
        presence = np.zeros((n_tirages, self.config.num_max))
        for i, draw in enumerate(historique):
            for num in draw:
                if 1 <= num <= self.config.num_max:
                    presence[i, num - 1] = 1
        
        for tirage in range(n_tirages):
            for num in range(self.config.num_max):
                # Combinaison linéaire des features (comme dans le code FreeBasic)
                val = (self.coef1 * presence[tirage, num] +
                       self.coef2 * freq_relative[num] +
                       self.coef3 * (dejavu[tirage, num] / 15.0) +  # Normalisation par 15
                       self.coef4 * moving_avg[tirage, num] +
                       self.coef5 * entropy_norm[num])
                
                # Contraindre entre 0 et 0.99
                X[tirage, num] = np.clip(val, 0, 0.99)
        
        # Calculer les cibles (fréquence cumulative normalisée)
        freq_cumul = np.zeros((n_tirages, self.config.num_max))
        for tirage in range(n_tirages):
            for draw in historique[:tirage+1]:
                for num in draw:
                    if 1 <= num <= self.config.num_max:
                        freq_cumul[tirage, num - 1] += 1
            
            total = np.sum(freq_cumul[tirage])
            if total > 0:
                y[tirage] = np.clip(freq_cumul[tirage] / total, 0, 0.99)
        
        return X, y


# ============================================================
#     MODULE NEURAL NETWORK POUR IA LABO
# ============================================================

class PredictionModule(ABC):
    """Interface commune pour tous les modules de prédiction."""
    
    def __init__(self, name: str, config: GameConfig):
        self.name = name
        self.config = config
        self.historique: List[List[int]] = []
        self.scores: Dict[int, float] = {}
        self.confidence: float = 1.0
        
    @abstractmethod
    def compute_scores(self) -> Dict[int, float]:
        pass
    
    def get_top_numbers(self, n: int) -> List[int]:
        sorted_nums = sorted(self.scores.items(), key=lambda x: x[1], reverse=True)
        return [num for num, score in sorted_nums[:n]]


class NeuralNetworkModule(PredictionModule):
    """
    Module de réseau de neurones avec entrées pondérées (Variante 1).
    Utilise SGD pour l'optimisation.
    """
    
    def __init__(self, config: GameConfig,
                 hidden_layers: List[int] = [128, 64],
                 epochs: int = 50,
                 learning_rate: float = 0.001,
                 coefficients: Tuple[float, ...] = (0.2, 0.25, 0.2, 0.15, 0.2)):
        
        super().__init__("NeuralNet_SGD", config)
        
        self.hidden_layers = hidden_layers
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.coefficients = coefficients
        
        # Initialiser le calculateur de features
        self.feature_calc = FeatureCalculator(
            config,
            coef1=coefficients[0],
            coef2=coefficients[1],
            coef3=coefficients[2],
            coef4=coefficients[3],
            coef5=coefficients[4]
        )
        
        # Le réseau sera initialisé lors du calcul des scores
        self.network = None
        self.training_losses = []
        
    def compute_scores(self) -> Dict[int, float]:
        """Entraîne le réseau et calcule les scores de sortie."""
        if len(self.historique) < 10:
            print(f"   ⚠️  {self.name}: Pas assez de données pour l'entraînement")
            return {i: 0.5 for i in range(1, self.config.num_max + 1)}
        
        # Calculer les features
        X, y = self.feature_calc.compute_all_features(self.historique)
        
        # Initialiser le réseau
        self.network = NeuralNetworkSGD(
            input_size=self.config.num_max,
            hidden_layers=self.hidden_layers,
            output_size=self.config.num_max,
            learning_rate=self.learning_rate
        )
        
        # Entraîner le réseau
        self.training_losses = self.network.train(X, y, epochs=self.epochs, verbose=False)
        
        # Prédiction finale (utiliser les dernières données)
        last_features = X[-1:, :]
        predictions = self.network.forward(last_features)
        
        # Convertir en scores
        output = predictions.flatten()
        for i in range(self.config.num_max):
            self.scores[i + 1] = float(output[i])
        
        return self.scores
    
    def get_training_loss(self) -> float:
        """Retourne la dernière perte d'entraînement."""
        if self.training_losses:
            return self.training_losses[-1]
        return float('inf')


# ============================================================
#              AUTRES MODULES (simplifiés)
# ============================================================

class FrequencyModule(PredictionModule):
    """Module basé sur les fréquences."""
    
    def __init__(self, config: GameConfig):
        super().__init__("Fréquence", config)
        
    def compute_scores(self) -> Dict[int, float]:
        if not self.historique:
            return {i: 0.0 for i in range(1, self.config.num_max + 1)}
        
        all_nums = [n for draw in self.historique for n in draw]
        freq = Counter(all_nums)
        
        max_freq = max(freq.values()) if freq else 1
        for i in range(1, self.config.num_max + 1):
            self.scores[i] = freq.get(i, 0) / max_freq
            
        return self.scores


class GapModule(PredictionModule):
    """Module basé sur les écarts."""
    
    def __init__(self, config: GameConfig):
        super().__init__("Écarts", config)
        
    def compute_scores(self) -> Dict[int, float]:
        if not self.historique:
            return {i: 0.0 for i in range(1, self.config.num_max + 1)}
        
        last_seen = {i: -1 for i in range(1, self.config.num_max + 1)}
        
        for idx, draw in enumerate(self.historique):
            for num in draw:
                last_seen[num] = idx
        
        last_idx = len(self.historique) - 1
        gaps = {}
        for num, seen_idx in last_seen.items():
            gaps[num] = (last_idx - seen_idx) if seen_idx != -1 else len(self.historique)
        
        max_gap = max(gaps.values()) if gaps else 1
        for num, gap in gaps.items():
            self.scores[num] = gap / max_gap
            
        return self.scores


class MarkovModule(PredictionModule):
    """Module basé sur les chaînes de Markov."""
    
    def __init__(self, config: GameConfig):
        super().__init__("Markov", config)
        
    def compute_scores(self) -> Dict[int, float]:
        if len(self.historique) < 2:
            return {i: 0.0 for i in range(1, self.config.num_max + 1)}
        
        # Matrice de transition
        transitions = Counter()
        for i in range(len(self.historique) - 1):
            for num in self.historique[i]:
                for next_num in self.historique[i + 1]:
                    transitions[(num, next_num)] += 1
        
        # Scores basés sur le dernier tirage
        last_draw = self.historique[-1]
        transition_scores = Counter()
        
        for num in last_draw:
            for (from_num, to_num), count in transitions.items():
                if from_num == num:
                    transition_scores[to_num] += count
        
        max_score = max(transition_scores.values()) if transition_scores else 1
        for i in range(1, self.config.num_max + 1):
            self.scores[i] = transition_scores.get(i, 0) / max_score
            
        return self.scores


# ============================================================
#              AGRÉGATEUR PRINCIPAL (IA LABO)
# ============================================================

class IALabo:
    """
    Système d'agrégation multi-modèles avec réseau de neurones.
    """
    
    def __init__(self, config: GameConfig):
        self.config = config
        self.historique: List[List[int]] = []
        
        # Initialiser les modules (incluant le nouveau NeuralNetwork)
        self.modules: List[PredictionModule] = [
            FrequencyModule(config),
            MarkovModule(config),
            GapModule(config),
            NeuralNetworkModule(config, 
                               hidden_layers=[128, 64],
                               epochs=100,
                               learning_rate=0.001),
        ]
        
        self.weights: Dict[str, float] = {m.name: 1.0 for m in self.modules}
        self._load_data()
        
    def _load_data(self):
        """Charge les données historiques."""
        try:
            df = pd.read_csv(self.config.filepath, sep=';')
            num_cols = [col for col in df.columns if col.startswith('num')][:self.config.nums_per_draw]
            if not num_cols:
                num_cols = df.select_dtypes(include=[np.number]).columns[:self.config.nums_per_draw].tolist()
            
            self.historique = df[num_cols].dropna().apply(
                lambda x: sorted([int(v) for v in x.tolist()]), axis=1
            ).tolist()
            
            for module in self.modules:
                module.historique = self.historique
                
            print(f"✅ {len(self.historique)} tirages chargés pour {self.config.name}")
            
        except FileNotFoundError:
            print(f"⚠️  Fichier '{self.config.filepath}' introuvable. Mode démo activé.")
            import random
            self.historique = [
                sorted(random.sample(range(1, self.config.num_max + 1), self.config.nums_per_draw))
                for _ in range(500)
            ]
            for module in self.modules:
                module.historique = self.historique
                
    def _compute_all_scores(self):
        """Calcule les scores de tous les modules."""
        print("\n⚙️  Calcul des scores par module...")
        for module in self.modules:
            module.compute_scores()
            if isinstance(module, NeuralNetworkModule):
                loss = module.get_training_loss()
                print(f"   ✓ {module.name} (loss finale: {loss:.6f})")
            else:
                print(f"   ✓ {module.name}")
            
    def aggregate_bagging(self) -> Dict[int, float]:
        """BAGGING: Moyenne simple des scores."""
        aggregated = {i: 0.0 for i in range(1, self.config.num_max + 1)}
        
        for num in aggregated:
            total = sum(module.scores.get(num, 0) for module in self.modules)
            aggregated[num] = total / len(self.modules)
            
        return aggregated
    
    def aggregate_boosting(self) -> Dict[int, float]:
        """BOOSTING: Moyenne pondérée."""
        aggregated = {i: 0.0 for i in range(1, self.config.num_max + 1)}
        total_weight = sum(self.weights.values())
        
        for num in aggregated:
            weighted_sum = sum(
                module.scores.get(num, 0) * self.weights[module.name]
                for module in self.modules
            )
            aggregated[num] = weighted_sum / total_weight
            
        return aggregated
    
    def generate_combinations(self, n: int = 5, method: str = "boosting") -> List[List[int]]:
        """Génère n combinaisons."""
        self._compute_all_scores()
        
        if method == "bagging":
            final_scores = self.aggregate_bagging()
        else:
            final_scores = self.aggregate_boosting()
        
        sorted_nums = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)
        
        combinations_list = []
        top_pool = [num for num, score in sorted_nums[:30]]
        
        for i in range(n):
            if i == 0:
                combo = [num for num, score in sorted_nums[:self.config.nums_per_draw]]
            else:
                weights = [final_scores[num] for num in top_pool]
                total_w = sum(weights)
                probs = [w/total_w for w in weights]
                
                combo = list(np.random.choice(
                    top_pool, 
                    size=self.config.nums_per_draw, 
                    replace=False,
                    p=probs
                ))
            
            combinations_list.append(sorted(combo))
            
        return combinations_list
    
    def display_neural_network_analysis(self):
        """Affiche une analyse détaillée du module réseau de neurones."""
        print("\n" + "="*70)
        print(f"{'🧠 ANALYSE RÉSEAU DE NEURONES (VARIANTE 1 - SGD)':^70}")
        print("="*70)
        
        # Trouver le module NN
        nn_module = None
        for m in self.modules:
            if isinstance(m, NeuralNetworkModule):
                nn_module = m
                break
        
        if nn_module is None:
            print("Module réseau de neurones non trouvé.")
            return
        
        # Afficher les coefficients
        print("\n📊 COEFFICIENTS DES FEATURES:")
        print(f"   • Données historiques (coef1): {nn_module.coefficients[0]}")
        print(f"   • Fréquence relative (coef2):  {nn_module.coefficients[1]}")
        print(f"   • Déjà vu / Écart (coef3):     {nn_module.coefficients[2]}")
        print(f"   • Moyenne mobile (coef4):      {nn_module.coefficients[3]}")
        print(f"   • Entropie (coef5):            {nn_module.coefficients[4]}")
        
        # Afficher l'architecture
        print("\n🏗️  ARCHITECTURE DU RÉSEAU:")
        print(f"   • Couche d'entrée:  {self.config.num_max} neurones")
        for i, size in enumerate(nn_module.hidden_layers):
            print(f"   • Couche cachée {i+1}: {size} neurones")
        print(f"   • Couche de sortie: {self.config.num_max} neurones")
        
        # Top prédictions
        print("\n🎯 TOP 20 PRÉDICTIONS DU RÉSEAU DE NEURONES:")
        top20 = nn_module.get_top_numbers(20)
        print(f"   {top20}")
        
        # Détail des scores
        print("\n📈 SCORES DÉTAILLÉS (Top 10):")
        sorted_scores = sorted(nn_module.scores.items(), key=lambda x: x[1], reverse=True)[:10]
        for num, score in sorted_scores:
            bar = "█" * int(score * 30)
            print(f"   N°{num:2d}: {score:.4f} {bar}")
    
    def display_full_analysis(self):
        """Affiche l'analyse complète."""
        print("\n" + "="*70)
        print(f"{'🔬 IA LABO - ANALYSE MULTI-MODÈLES':^70}")
        print(f"{'Jeu: ' + self.config.name:^70}")
        print("="*70)
        
        self._compute_all_scores()
        
        # Top 10 de chaque module
        print("\n📊 TOP 10 PAR MODULE:")
        print("-"*70)
        for module in self.modules:
            top10 = module.get_top_numbers(10)
            print(f"   {module.name:15} → {top10}")
        
        # Analyse détaillée du réseau de neurones
        self.display_neural_network_analysis()
        
        # Résultats agrégés
        print("\n" + "="*70)
        print("📦 RÉSULTATS AGRÉGÉS")
        print("-"*70)
        
        bagging_scores = self.aggregate_bagging()
        bagging_top = sorted(bagging_scores.items(), key=lambda x: x[1], reverse=True)[:15]
        print(f"\n   BAGGING (moyenne):    {[num for num, _ in bagging_top]}")
        
        boosting_scores = self.aggregate_boosting()
        boosting_top = sorted(boosting_scores.items(), key=lambda x: x[1], reverse=True)[:15]
        print(f"   BOOSTING (pondéré):   {[num for num, _ in boosting_top]}")
        
        # Combinaisons suggérées
        print("\n" + "="*70)
        print("🎰 COMBINAISONS SUGGÉRÉES")
        print("-"*70)
        
        combos = self.generate_combinations(5, method="boosting")
        for i, combo in enumerate(combos, 1):
            print(f"   Grille {i}: {combo} (somme: {sum(combo)})")
        
        print("\n" + "="*70)
        print("="*70)


# ============================================================
#                      EXÉCUTION
# ============================================================

if __name__ == "__main__":
    print("""
    ╔══════════════════════════════════════════════════════════════════╗
    ║                                                                  ║
    ║          🧠 IA LABO - VARIANTE 1 (RÉSEAU DE NEURONES) 🧠         ║
    ║                    Entrées Pondérées + SGD                       ║
    ║                                                                  ║
    ╚══════════════════════════════════════════════════════════════════╝
    """)
    
    # Configuration (changer pour votre jeu)
    config = CONFIGS["loto"]  # ou "loto", "euromillions"
    
    try:
        lab = IALabo(config)
        lab.display_full_analysis()
        
    except Exception as e:
        print(f"❌ Erreur: {e}")
        import traceback
        traceback.print_exc()


    ╔══════════════════════════════════════════════════════════════════╗
    ║                                                                  ║
    ║          🧠 IA LABO - VARIANTE 1 (RÉSEAU DE NEURONES) 🧠         ║
    ║                    Entrées Pondérées + SGD                       ║
    ║                                                                  ║
    ╚══════════════════════════════════════════════════════════════════╝
    
✅ 960 tirages chargés pour Loto

                  🔬 IA LABO - ANALYSE MULTI-MODÈLES                   
                              Jeu: Loto                               

⚙️  Calcul des scores par module...
   ✓ Fréquence
   ✓ Markov
   ✓ Écarts
   ✓ NeuralNet_SGD (loss finale: 0.426809)

📊 TOP 10 PAR MODULE:
----------------------------------------------------------------------
   Fréquence       → [31, 6, 15, 3, 24, 26, 22, 28, 7, 30]
   Markov          → [15, 3, 26, 17, 6, 4, 30, 31, 44, 7]
   Écarts          → [47, 21, 46, 45, 20, 32, 36, 5, 3